In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
# import matplotlib.pyplot as plt
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import AdaBoostClassifier
from google.colab import files
from geopy.geocoders import Nominatim
from sklearn import preprocessing
import itertools

# Get the Data from Kaggle

Credit to Kaggle user Filemon [1] for the utility code used here to get our dataset from Kaggle. It was copied and pasted with some minor alterations. Prof. Numanagic confirmed on Piazza that using this utility code is fine so long as we cite our source.

In [ ]:
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list
! kaggle datasets download -d ruchi798/housing-prices-in-metropolitan-areas-of-india -p /content/sample_data/ --unzip

Saving kaggle.json to kaggle.json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              233KB  2021-08-12 07:39:36          10578  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           6463  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           2421  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   2GB  2021-07-03 18:37:20

In [ ]:
cities = ['Bangalore', 'Chennai', 'Delhi', 'Hyderabad', 'Kolkata', 'Mumbai']
df1 = pd.read_csv("/content/sample_data/Bangalore.csv")
df2 = pd.read_csv("/content/sample_data/Chennai.csv")
df3 = pd.read_csv("/content/sample_data/Delhi.csv")
df4 = pd.read_csv("/content/sample_data/Hyderabad.csv")
df5 = pd.read_csv("/content/sample_data/Kolkata.csv")
df6 = pd.read_csv("/content/sample_data/Mumbai.csv")
df1.head()

,Price,Area,Location,No. of Bedrooms,Resale,MaintenanceStaff,Gymnasium,SwimmingPool,LandscapedGardens,JoggingTrack,RainWaterHarvesting,IndoorGames,ShoppingMall,Intercom,SportsFacility,ATM,ClubHouse,School,24X7Security,PowerBackup,CarParking,StaffQuarter,Cafeteria,MultipurposeRoom,Hospital,WashingMachine,Gasconnection,AC,Wifi,Children'splayarea,LiftAvailable,BED,VaastuCompliant,Microwave,GolfCourse,TV,DiningTable,Sofa,Wardrobe,Refrigerator
0,30000000,3340,JP Nagar Phase 1,4,0,1,1,1,1,1,1,1,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
1,7888000,1045,Dasarahalli on Tumkur Road,2,0,0,1,1,1,1,1,1,0,0,1,0,1,0,1,1,1,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0
2,4866000,1179,Kannur on Thanisandra Main Road,2,0,0,1,1,1,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
3,8358000,1675,Doddanekundi,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,6845000,1670,Kengeri,3,0,1,1,1,1,1,1,1,0,1,1,0,1,0,1,1,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0


In [ ]:
dfs = [df1, df2, df3, df4, df5, df6]
for (i,k) in zip(dfs,cities):
  print(f"{k:<10} shape: {i.shape}")

Bangalore  shape: (6207, 40)
Chennai    shape: (5014, 40)
Delhi      shape: (4998, 40)
Hyderabad  shape: (2518, 40)
Kolkata    shape: (6507, 40)
Mumbai     shape: (7719, 40)


In [ ]:
dfs[0].describe()

,Price,Area,No. of Bedrooms,Resale,MaintenanceStaff,Gymnasium,SwimmingPool,LandscapedGardens,JoggingTrack,RainWaterHarvesting,IndoorGames,ShoppingMall,Intercom,SportsFacility,ATM,ClubHouse,School,24X7Security,PowerBackup,CarParking,StaffQuarter,Cafeteria,MultipurposeRoom,Hospital,WashingMachine,Gasconnection,AC,Wifi,Children'splayarea,LiftAvailable,BED,VaastuCompliant,Microwave,GolfCourse,TV,DiningTable,Sofa,Wardrobe,Refrigerator
count,6.207000e+03,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000,6207.000000
mean,1.058510e+07,1526.094248,2.556952,0.078782,6.208797,6.461576,6.436121,6.382471,6.396649,6.373933,6.348639,6.207024,6.420010,6.332206,6.210246,6.397293,6.208152,6.394071,6.455292,6.353633,6.240696,6.236668,6.345416,6.188980,6.210569,6.233768,6.211374,6.171097,6.453037,6.438537,6.211052,6.311100,6.210569,6.181569,6.210569,6.210569,6.210569,6.171097,6.210569
std,1.410943e+07,764.845609,0.694300,0.269420,4.126883,3.752421,3.792567,3.875271,3.853661,3.888204,3.926154,4.129334,3.817671,3.950528,4.124876,3.852674,4.127774,3.857603,3.762386,3.918703,4.082386,4.088044,3.930951,4.154162,4.124430,4.092111,4.123314,4.178546,3.765955,3.788782,4.123761,3.981513,4.124430,4.164294,4.124430,4.124430,4.124430,4.178546,4.124430
min,2.000000e+06,415.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000e+06,1110.000000,2.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.368000e+06,1340.000000,3.000000,0.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
75%,1.070000e+07,1662.500000,3.000000,0.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
max,3.000000e+08,9900.000000,7.000000,1.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000


In [ ]:
dfs[1].describe()

,Price,Area,No. of Bedrooms,Resale,MaintenanceStaff,Gymnasium,SwimmingPool,LandscapedGardens,JoggingTrack,RainWaterHarvesting,IndoorGames,ShoppingMall,Intercom,SportsFacility,ATM,ClubHouse,School,24X7Security,PowerBackup,CarParking,StaffQuarter,Cafeteria,MultipurposeRoom,Hospital,WashingMachine,Gasconnection,AC,Wifi,Children'splayarea,LiftAvailable,BED,VaastuCompliant,Microwave,GolfCourse,TV,DiningTable,Sofa,Wardrobe,Refrigerator
count,5.014000e+03,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000,5014.000000
mean,8.731264e+06,1232.992022,2.362984,0.097128,5.000997,5.213602,5.160949,5.114878,5.123055,5.115277,5.120463,5.008576,5.119266,5.080574,5.028121,5.154966,5.017351,5.121859,5.253889,5.121260,5.019944,5.037296,5.084164,5.009374,4.999601,5.017750,4.996410,4.991823,5.202433,5.264260,4.999402,5.068010,4.997208,4.991823,5.009174,5.011767,4.998404,4.991823,5.009174
std,1.034261e+07,641.841841,0.705252,0.296162,4.464256,4.239118,4.296954,4.346406,4.337706,4.345983,4.340468,4.456600,4.341742,4.382551,4.436736,4.303437,4.447703,4.338981,4.193880,4.339619,4.445068,4.427352,4.378795,4.455793,4.465663,4.447298,4.468877,4.473489,4.251507,4.182092,4.465864,4.395648,4.468074,4.473489,4.455995,4.453369,4.466869,4.473489,4.455995
min,2.000000e+06,362.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.095000e+06,865.000000,2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.782000e+06,1067.000000,2.000000,0.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
75%,8.675000e+06,1438.750000,3.000000,0.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
max,1.900000e+08,7240.000000,5.000000,1.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000


In [ ]:
dfs[2].describe()

,Price,Area,No. of Bedrooms,Resale,MaintenanceStaff,Gymnasium,SwimmingPool,LandscapedGardens,JoggingTrack,RainWaterHarvesting,IndoorGames,ShoppingMall,Intercom,SportsFacility,ATM,ClubHouse,School,24X7Security,PowerBackup,CarParking,StaffQuarter,Cafeteria,MultipurposeRoom,Hospital,WashingMachine,Gasconnection,AC,Wifi,Children'splayarea,LiftAvailable,BED,VaastuCompliant,Microwave,GolfCourse,TV,DiningTable,Sofa,Wardrobe,Refrigerator
count,4.998000e+03,4998.000000,4998.000000,4998.00000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.00000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.00000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000
mean,1.588638e+07,1277.343737,2.657863,0.80012,5.423770,5.472589,5.415966,5.435574,5.412765,5.437975,5.398359,5.398960,5.570228,5.487795,5.39916,5.460184,5.399560,5.423770,5.530812,5.490996,5.407163,5.399360,5.402961,5.39916,5.408363,5.562225,5.426971,5.394958,5.534014,5.655662,5.416367,5.456783,5.413165,5.394958,5.415566,5.408764,5.412965,5.394958,5.411565
std,4.612894e+07,915.217800,0.820605,0.39995,4.378355,4.322819,4.387116,4.365041,4.390702,4.362324,4.406770,4.406103,4.207849,4.305261,4.40588,4.337050,4.405435,4.378355,4.254905,4.301549,4.396963,4.405658,4.401649,4.40588,4.395623,4.217476,4.374752,4.410549,4.251116,4.102704,4.386668,4.340938,4.390254,4.410549,4.387565,4.395176,4.390478,4.410549,4.392045
min,2.000000e+06,200.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.900000e+06,800.000000,2.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.500000e+06,1000.000000,3.000000,1.00000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.00000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.00000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
75%,1.580000e+07,1650.000000,3.000000,1.00000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.00000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.00000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
max,8.546000e+08,16000.000000,8.000000,1.00000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.00000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.00000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000


In [ ]:
dfs[3].describe()

,Price,Area,No. of Bedrooms,Resale,MaintenanceStaff,Gymnasium,SwimmingPool,LandscapedGardens,JoggingTrack,RainWaterHarvesting,IndoorGames,ShoppingMall,Intercom,SportsFacility,ATM,ClubHouse,School,24X7Security,PowerBackup,CarParking,StaffQuarter,Cafeteria,MultipurposeRoom,Hospital,WashingMachine,Gasconnection,AC,Wifi,Children'splayarea,LiftAvailable,BED,VaastuCompliant,Microwave,GolfCourse,TV,DiningTable,Sofa,Wardrobe,Refrigerator
count,2.518000e+03,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000
mean,9.818380e+06,1644.571088,2.623511,0.227562,0.412629,0.739873,0.709690,0.619142,0.559968,0.606434,0.580620,0.366164,0.679110,0.553614,0.421763,0.711676,0.340747,0.706513,0.935663,0.736299,0.395552,0.413423,0.532168,0.339952,0.320095,0.386815,0.338761,0.326052,0.754170,0.975774,0.360207,0.607228,0.342732,0.330024,0.321287,0.334392,0.342335,0.341144,0.324861
std,8.777113e+06,746.256956,0.687330,0.419341,1.626445,1.611024,1.615231,1.624426,1.627686,1.625308,1.626794,1.623281,1.618909,1.627908,1.626910,1.614972,1.620984,1.615641,1.569421,1.611552,1.625437,1.626487,1.628473,1.620905,1.618821,1.624852,1.620787,1.619472,1.608829,1.557725,1.622778,1.625256,1.621178,1.619894,1.618953,1.620347,1.621139,1.621023,1.619344
min,2.000000e+06,500.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.760000e+06,1160.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.754000e+06,1500.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.090000e+07,1829.000000,3.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.650000e+08,9400.000000,8.000000,1.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000


In [ ]:
dfs[4].describe()

,Price,Area,No. of Bedrooms,Resale,MaintenanceStaff,Gymnasium,SwimmingPool,LandscapedGardens,JoggingTrack,RainWaterHarvesting,IndoorGames,ShoppingMall,Intercom,SportsFacility,ATM,ClubHouse,School,24X7Security,PowerBackup,CarParking,StaffQuarter,Cafeteria,MultipurposeRoom,Hospital,WashingMachine,Gasconnection,AC,Wifi,Children'splayarea,LiftAvailable,BED,VaastuCompliant,Microwave,GolfCourse,TV,DiningTable,Sofa,Wardrobe,Refrigerator
count,6.507000e+03,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000,6507.000000
mean,8.508570e+06,1344.166897,2.631166,0.311818,8.899032,8.903028,8.901644,8.901644,8.900415,8.899493,8.900876,8.897188,8.901798,8.900108,8.896573,8.901491,8.896419,8.903489,8.904411,8.901491,8.897649,8.898417,8.898571,8.897034,8.897188,8.896727,8.897034,8.896266,8.902413,8.905025,8.897034,8.899339,8.897034,8.896727,8.896727,8.896880,8.897034,8.896266,8.897034
std,1.465939e+07,807.698043,0.712283,0.463272,0.936229,0.899654,0.912482,0.912482,0.923734,0.932083,0.919531,0.952630,0.911066,0.926525,0.958034,0.913897,0.959380,0.895337,0.886638,0.913897,0.948556,0.941728,0.940356,0.953984,0.952630,0.956686,0.953984,0.960724,0.905378,0.880791,0.953984,0.933467,0.953984,0.956686,0.956686,0.955336,0.953984,0.960724,0.953984
min,2.000000e+06,350.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.300000e+06,920.000000,2.000000,0.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
50%,4.940000e+06,1144.000000,3.000000,0.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
75%,8.000000e+06,1454.000000,3.000000,1.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
max,5.040000e+08,10000.000000,9.000000,1.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000


In [ ]:
dfs[5].describe()

,Price,Area,No. of Bedrooms,Resale,MaintenanceStaff,Gymnasium,SwimmingPool,LandscapedGardens,JoggingTrack,RainWaterHarvesting,IndoorGames,ShoppingMall,Intercom,SportsFacility,ATM,ClubHouse,School,24X7Security,PowerBackup,CarParking,StaffQuarter,Cafeteria,MultipurposeRoom,Hospital,WashingMachine,Gasconnection,AC,Wifi,Children'splayarea,LiftAvailable,BED,VaastuCompliant,Microwave,GolfCourse,TV,DiningTable,Sofa,Wardrobe,Refrigerator
count,7.719000e+03,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000,7719.000000
mean,1.506165e+07,998.409250,1.913331,0.647105,7.498899,7.473896,7.437881,7.441638,7.439435,7.477005,7.404845,7.402772,7.436067,7.395777,7.404586,7.450188,7.402902,7.537505,7.523254,7.538800,7.426221,7.399275,7.402902,7.402254,7.373105,7.395258,7.382044,7.387097,7.434512,7.518331,7.417930,7.454722,7.372069,7.379712,7.379065,7.373624,7.374530,7.383988,7.374789
std,2.052100e+07,550.967809,0.855376,0.477901,3.197923,3.252095,3.328245,3.320401,3.325002,3.245418,3.396259,3.400471,3.332024,3.414637,3.396786,3.302464,3.400208,3.112033,3.144066,3.109101,3.352445,3.407563,3.400208,3.401522,3.460050,3.415683,3.442233,3.432112,3.335259,3.155041,3.369523,3.292904,3.462108,3.446892,3.448185,3.459020,3.457217,3.438345,3.456702
min,2.000000e+06,200.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.300000e+06,650.000000,1.000000,0.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
50%,9.500000e+06,900.000000,2.000000,1.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
75%,1.700000e+07,1177.000000,2.000000,1.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
max,4.200000e+08,8511.000000,7.000000,1.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000


Change the name for Children'splayarea to KidsPlayArea

In [ ]:
dfs = [df1, df2, df3, df4, df5, df6]
for i, k in enumerate(dfs):
  dfs[i] = k.rename(columns={"Children'splayarea":"KidsPlayArea"})

Credit to Kaggle user Pradyut Lohani [2] for identifiying and replacing "9" values with NaNs. "9" was used in the original dataset to mention absence of information; by replacing 9s with NaNs, we can use data that has some missing values without it throwing off our classifier.

In [ ]:
for i, k in enumerate(dfs):
  k.replace(9,np.nan, inplace=True)

We select the columns which have missing data to either replace them with simple imputation or delete samples with missing data

In [ ]:
col_names = dfs[0].columns
col_full_vals = ['Price', 'Area', 'Location', 'No. of Bedrooms', 'Resale']
col_not_full_vals = [col_names for col_names in col_names if col_names not in col_full_vals]

**Uncomment the following it to get datasets with simple imputation**

In [ ]:
for j in dfs:
  for i in col_not_full_vals:
    j[i].fillna(j[i].mean(), inplace = True)

In [ ]:
dfs[0].isna().sum() # count of how many properties are missing each type of data for Bangalore

Price                  0
Area                   0
Location               0
No. of Bedrooms        0
Resale                 0
MaintenanceStaff       0
Gymnasium              0
SwimmingPool           0
LandscapedGardens      0
JoggingTrack           0
RainWaterHarvesting    0
IndoorGames            0
ShoppingMall           0
Intercom               0
SportsFacility         0
ATM                    0
ClubHouse              0
School                 0
24X7Security           0
PowerBackup            0
CarParking             0
StaffQuarter           0
Cafeteria              0
MultipurposeRoom       0
Hospital               0
WashingMachine         0
Gasconnection          0
AC                     0
Wifi                   0
KidsPlayArea           0
LiftAvailable          0
BED                    0
VaastuCompliant        0
Microwave              0
GolfCourse             0
TV                     0
DiningTable            0
Sofa                   0
Wardrobe               0
Refrigerator           0


In [ ]:
            # def rename_location(dfs):
            #   for p in dfs:
            #     a = p['Location'].tolist()
            #     a_sorted = sorted(list(set(a)), key=str.lower)
            #     b = {}
            #     t = 0
            #     for i in a_sorted:
            #       b[i] = t
            #       t = t+1
            #     p['LocationRename'] = p['Location'].replace(b)

In [ ]:
            # dfs = [df1, df2, df3, df4, df5, df6]
            # rename_location(dfs)

In [ ]:
            # df1 = pd.get_dummies(df1)
            # df2 = pd.get_dummies(df2)
            # df3 = pd.get_dummies(df3)
            # df4 = pd.get_dummies(df4)
            # df5 = pd.get_dummies(df5)
            # df6 = pd.get_dummies(df6)

In [ ]:
for (i,k) in zip(dfs,cities):
  print(f"{k:<10} shape: {i.shape}")

Bangalore  shape: (6207, 40)
Chennai    shape: (5014, 40)
Delhi      shape: (4998, 40)
Hyderabad  shape: (2518, 40)
Kolkata    shape: (6507, 40)
Mumbai     shape: (7719, 40)


In [ ]:
geolocator = Nominatim(user_agent="housing")

def findLongLat(local):
  try:
    cordinates=geolocator.geocode(local)
    return cordinates.latitude, cordinates.longitude
  except:
    return "NA", "NA"

In [ ]:
def locationFindAndReplace(dfs):
  for j, (i, k) in enumerate(zip(dfs, cities)):
    df_selection = i
    city_selection = k
    lolk= df_selection['Location'].unique()
    lolp = [ str(t) +', '+str(city_selection) +', India' for t in lolk]
    locationLongLat={}
    print(f'{i.shape}, {city_selection}')
    for p, place in zip(lolk, lolp):
      locationLongLat[p]=findLongLat(place)
      print(place)
    dfs[j]['Latitude'], dfs[j]['Longitude'] = zip(*dfs[j]['Location'].map(locationLongLat))
    dfs[j].replace('NA', np.nan, inplace=True)
    dfs[j] = dfs[j].drop(columns='Location')
  return dfs

In [ ]:
dfs = locationFindAndReplace(dfs)

(6207, 40), Bangalore
JP Nagar Phase 1, Bangalore, India
Dasarahalli on Tumkur Road, Bangalore, India
Kannur on Thanisandra Main Road, Bangalore, India
Doddanekundi, Bangalore, India
Kengeri, Bangalore, India
Horamavu, Bangalore, India
Thanisandra, Bangalore, India
Ramamurthy Nagar, Bangalore, India
Whitefield Hope Farm Junction, Bangalore, India
Electronic City Phase 1, Bangalore, India
Yelahanka, Bangalore, India
Anjanapura, Bangalore, India
Jalahalli, Bangalore, India
Kasavanahalli, Bangalore, India
Bommasandra, Bangalore, India
Bellandur, Bangalore, India
RR Nagar, Bangalore, India
Begur, Bangalore, India
Hosa Road, Bangalore, India
Sahakar Nagar, Bangalore, India
Kadugodi, Bangalore, India
Jakkur, Bangalore, India
Jigani, Bangalore, India
Krishnarajapura, Bangalore, India
Brookefield, Bangalore, India
Banashankari, Bangalore, India
Nelamangala, Bangalore, India
Attibele, Bangalore, India
Banaswadi, Bangalore, India
Kodigehalli, Bangalore, India
ITPL, Bangalore, India
Uttarahalli H

In [ ]:
# dfs[0]['Latitude'], dfs[0]['Longitude'] = zip(*dfs[0]['Location'].map(locationLongLat))
# df1['Latitude'],df1['Longitude']=zip(*df1['Location'].map(locationLongLat))
# df2['Latitude'],df2['Longitude']=zip(*df2['Location'].map(locationLongLat))
# df3['Latitude'],df3['Longitude']=zip(*df3['Location'].map(locationLongLat))
# df4['Latitude'],df4['Longitude']=zip(*df4['Location'].map(locationLongLat))
# df5['Latitude'],df5['Longitude']=zip(*df5['Location'].map(locationLongLat))
# df6['Latitude'],df6['Longitude']=zip(*df6['Location'].map(locationLongLat))
# # convertLocation(df4)

In [ ]:
# for i in dfs:
#   i.replace('NA', np.nan, inplace=True)
# df1.replace('NA', np.nan, inplace=True)
# df2.replace('NA', np.nan, inplace=True)
# df3.replace('NA', np.nan, inplace=True)
# df4.replace('NA', np.nan, inplace=True)
# df5.replace('NA', np.nan, inplace=True)
# df6.replace('NA', np.nan, inplace=True)

In [ ]:
for (i,k) in zip(dfs,cities):
  print(f"{k:<10} shape: {i.shape}")

Bangalore  shape: (6207, 41)
Chennai    shape: (5014, 41)
Delhi      shape: (4998, 41)
Hyderabad  shape: (2518, 41)
Kolkata    shape: (6507, 41)
Mumbai     shape: (7719, 41)


Total sample with missing Values; after removing missing location values

*   Bangalore: 4256 of 6207: 1951: 1530
*   Chennai: 2781 of 5014: 2233: 1917
*   Delhi: 2996 of 4998: 2002: 1949
*   Hyderabad: 84 of 2518: 2434: 2174
*   Kolkata: 6507 of 6432: 75: 65
*   Mumbai: 6321 of 7719: 1398: 1114

Imputation:
* Bangalore  shape: (4914, 41)
* Chennai    shape: (4287, 41)
* Delhi      shape: (4747, 41)
* Hyderabad  shape: (2251, 41)
* Kolkata    shape: (5553, 41)
* Mumbai     shape: (6806, 41)

In [ ]:
for i in dfs:
  i.dropna(inplace=True)
# df1.dropna(inplace= True)
# df2.dropna(inplace= True)
# df3.dropna(inplace= True)
# df4.dropna(inplace= True)
# df5.dropna(inplace= True)
# df6.dropna(inplace= True)

In [ ]:
for (i,k) in zip(dfs,cities):
  print(f"{k:<10} shape: {i.shape}")

Bangalore  shape: (4914, 41)
Chennai    shape: (4287, 41)
Delhi      shape: (4747, 41)
Hyderabad  shape: (2251, 41)
Kolkata    shape: (5553, 41)
Mumbai     shape: (6806, 41)


In [ ]:
dfs[0].isna().sum()

Price                  0
Area                   0
No. of Bedrooms        0
Resale                 0
MaintenanceStaff       0
Gymnasium              0
SwimmingPool           0
LandscapedGardens      0
JoggingTrack           0
RainWaterHarvesting    0
IndoorGames            0
ShoppingMall           0
Intercom               0
SportsFacility         0
ATM                    0
ClubHouse              0
School                 0
24X7Security           0
PowerBackup            0
CarParking             0
StaffQuarter           0
Cafeteria              0
MultipurposeRoom       0
Hospital               0
WashingMachine         0
Gasconnection          0
AC                     0
Wifi                   0
KidsPlayArea           0
LiftAvailable          0
BED                    0
VaastuCompliant        0
Microwave              0
GolfCourse             0
TV                     0
DiningTable            0
Sofa                   0
Wardrobe               0
Refrigerator           0
Latitude               0


In [ ]:
# df1 = df1.drop(columns='Location')
# df2 = df2.drop(columns='Location')
# df3 = df3.drop(columns='Location')
# df4 = df4.drop(columns='Location')
# df5 = df5.drop(columns='Location')
# df6 = df6.drop(columns='Location')

In [ ]:
# dfs = [df1, df2, df3, df4, df5, df6]
for i, k in enumerate(dfs):
  scalar = preprocessing.MinMaxScaler()
  x = k.values
  x_scaled = scalar.fit_transform(x)
  dfs[i] = pd.DataFrame(x_scaled, columns=k.columns)

In [ ]:
dfs[0].head()

,Price,Area,No. of Bedrooms,Resale,MaintenanceStaff,Gymnasium,SwimmingPool,LandscapedGardens,JoggingTrack,RainWaterHarvesting,IndoorGames,ShoppingMall,Intercom,SportsFacility,ATM,ClubHouse,School,24X7Security,PowerBackup,CarParking,StaffQuarter,Cafeteria,MultipurposeRoom,Hospital,WashingMachine,Gasconnection,AC,Wifi,KidsPlayArea,LiftAvailable,BED,VaastuCompliant,Microwave,GolfCourse,TV,DiningTable,Sofa,Wardrobe,Refrigerator,Latitude,Longitude
0,0.093960,0.308382,0.500000,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.034778,0.293278
1,0.021336,0.132841,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.034180,0.303187
2,0.016258,0.132314,0.333333,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.030072,0.283146
3,0.016097,0.084871,0.166667,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.038469,0.299765
4,0.060403,0.220032,0.500000,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040565,0.297294


In [ ]:
dfs[0].to_csv('BangaloreDataMunging.csv')
dfs[1].to_csv('ChennaiDataMunging.csv')
dfs[2].to_csv('DelhiDataMunging.csv')
dfs[3].to_csv('HyderabadDataMunging.csv')
dfs[4].to_csv('KolkataDataMunging.csv')
dfs[5].to_csv('MumbaiDataMunging.csv')

# Citations

[1] https://www.kaggle.com/general/74235

[2] https://www.kaggle.com/ruchi798/housing-prices-eda-and-prediction
